# CONTENT  BASED RECOMMENDATION SYSTEM

In [1]:
import pandas as pd
# for creating and using tabular datframes

In [2]:
# reading the movies.csv and ratings.csv from the movielens datset
movies = pd.read_csv('movies.csv')  
rating = pd.read_csv('ratings.csv') 

In [3]:
movies.shape
# seeing the form of our data

(9742, 3)

In [4]:
movies.head()
# displaying the movie dataset

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
rating.head()
# viewing the ratings dataset

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [6]:
rating.shape

(26024289, 4)

In [7]:
movies['year'] = movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies.head(3)
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles 

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,(1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,(1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,(1995)


In [8]:
# Removing the paranthesis fromt the year table 
movies['year'] = movies.year.str.extract('(\d\d\d\d)',expand=False)
movies.head(3)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995


In [9]:
# we are removing the year from the title of the movie
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')
movies.head(3)

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995


In [10]:
# using the strip function present in the pandas library to remove any extra white spaces in the end of the title
movies['title'] = movies['title'].apply(lambda x: x.strip())
movies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [11]:
# We also split the values in the Genres column into a list of Genres 
# This can be achieved by applying Python's split string function on the correct column.
# as the generes are separated by '|' we are using it as a separater for the split function
movies['genres'] = movies.genres.str.split('|')
movies.head()
# we are doing this as list data is more easy to use than strings

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [13]:
# dispalying the summary of the dataset
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 4 columns):
movieId    9742 non-null int64
title      9742 non-null object
genres     9742 non-null object
year       9729 non-null object
dtypes: int64(1), object(3)
memory usage: 304.5+ KB


In [14]:
movies_df_original_mem = movies.memory_usage()
movies_df_original_mem

Index         80
movieId    77936
title      77936
genres     77936
year       77936
dtype: int64

In [16]:
# Let's convert movieId column from int64 to int8 to save memory space
# This is necessary as the original dataset is big and processing it is difficult
movies.movieId = movies.movieId.astype('int32')

In [17]:
movies.isna().sum()
# there are 13 missing attributes in year of the movie dataset

movieId     0
title       0
genres      0
year       13
dtype: int64

In [18]:
movies.year.fillna(0, inplace=True)
# we are filling the missing years with zero

In [19]:
# we are converting the year from int64 to int8 as it holds a maximum a 4 digit number hence we can reduce space
movies.year = movies.year.astype('int16')

In [21]:
# we can see that these small changes have changed the size to almost half for year data
movies_new_mem = movies.memory_usage()
print(movies_df_original_mem)
print()
print(movies_new_mem)

Index         80
movieId    77936
title      77936
genres     77936
year       77936
dtype: int64

Index         80
movieId    38968
title      77936
genres     77936
year       19484
dtype: int64


In [22]:
movies.dtypes

movieId     int32
title      object
genres     object
year        int16
dtype: object

In [23]:
movies.head(3)

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995


we are One-Hot-Encoding the list of genres. we store every different genre in columns that contain either 1 or 0.

1 shows that a movie has that genre and 0 shows that it doesn't.
we are storing this dataframe in another variable, just incase we need the dataframe without genres.

In [25]:
movies_with_genres = movies.copy(deep=True) # making a copy of the movies dataframe

# Let's iterate through movies, then append the movie genres as columns of 1s or 0s.
# 1 if that column contains movies in the genre at the present index and 0 if not.

x = []
for index, row in movies.iterrows():
    x.append(index)
    for genre in row['genres']:
        movies_with_genres.at[index, genre] = 1
        
#checking if all the rows have been iterated as the matrix size becomes siilar to movies dataframe
print(len(x) == len(movies))

movies_with_genres.head(3)

True


,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,NaN,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# we can fill all the NAN values to zero to indicate the movie does not belong to that genre
movies_with_genres = movies_with_genres.fillna(0)
movies_with_genres.head(3)

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
rating.shape          
rating.head()
# viewing the ratings dataset

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [29]:
# dropping the timestamp column as it is not required for us
rating.drop('timestamp', axis=1, inplace=True)

# confirming if it has been dropped
rating.head(3)

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0


In [30]:
# summary of the rating dataset 
rating.dtypes

userId       int64
movieId      int64
rating     float64
dtype: object

In [85]:
rating.isna().sum() # we can see there are no missing attributs so we can proceed with this data

userId     0
movieId    0
rating     0
dtype: int64

We are creating a user Srija and assume she has rated a few movies now we are going to understand her movie interest and make a recommendation for her

In [69]:
# Ratings given by Srija to different movies on a scale of 0 to 5
Srija_movie_ratings = [
            {'title':'Predator', 'rating':4.9},
            {'title':'Final Destination', 'rating':4.9},
            {'title':'Mission Impossible', 'rating':4},
            {'title':"Beverly Hills Cop", 'rating':3},
            {'title':'Exorcist, The', 'rating':4.8},
            {'title':'Waiting to Exhale', 'rating':3.9},
            {'title':'Avengers, The', 'rating':4.5},
            {'title':'Omen, The', 'rating':5.0}
         ] 
Srija_movie_ratings = pd.DataFrame(Srija_movie_ratings)
Srija_movie_ratings

,rating,title
0,4.9,Predator
1,4.9,Final Destination
2,4.0,Mission Impossible
3,3.0,Beverly Hills Cop
4,4.8,"Exorcist, The"
5,3.9,Waiting to Exhale
6,4.5,"Avengers, The"
7,5.0,"Omen, The"


In [70]:
# adding movieId to the user rated data
Srija_movie_Id = movies[movies['title'].isin(Srija_movie_ratings['title'])]

#merging the Srija_movie_Id dataframe to the Srija_ratings dataframe it atomatically merges title and genres and year
Srija_movie_ratings = pd.merge(Srija_movie_Id, Srija_movie_ratings)

Srija_movie_ratings

,movieId,title,genres,year,rating
0,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,3.9
1,1350,"Omen, The","[Horror, Mystery, Thriller]",1976,5.0
2,45662,"Omen, The","[Horror, Thriller]",2006,5.0
3,1997,"Exorcist, The","[Horror, Mystery]",1973,4.8
4,2153,"Avengers, The","[Action, Adventure]",1998,4.5
5,89745,"Avengers, The","[Action, Adventure, Sci-Fi, IMAX]",2012,4.5
6,3409,Final Destination,"[Drama, Thriller]",2000,4.9
7,3527,Predator,"[Action, Sci-Fi, Thriller]",1987,4.9
8,4085,Beverly Hills Cop,"[Action, Comedy, Crime, Drama]",1984,3.0


In [71]:
#Srija_movie_ratings.reset_index(drop=True, inplace=True)
# we are fropping genres and year as we do not need them 
Srija_movie_ratings = Srija_movie_ratings.drop(["genres", "year"],1)

# final form of the input dataframe 
# if any movies given by user are not present here then it means they may not be available in original 
#or could be spelled differnt 
Srija_movie_ratings

,movieId,title,rating
0,4,Waiting to Exhale,3.9
1,1350,"Omen, The",5.0
2,45662,"Omen, The",5.0
3,1997,"Exorcist, The",4.8
4,2153,"Avengers, The",4.5
5,89745,"Avengers, The",4.5
6,3409,Final Destination,4.9
7,3527,Predator,4.9
8,4085,Beverly Hills Cop,3.0


In [72]:
#learning Srija's preferences
#so let's get the subset of movies that Srija has watched from the Dataframe containing genres defined with binary values.

# filtering the movies present in both srija's ratings and movie_with_genres 
Srija_genres = movies_with_genres[movies_with_genres.movieId.isin(Srija_movie_ratings.movieId)]
Srija_genres

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1038,1350,"Omen, The","[Horror, Mystery, Thriller]",1976,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1472,1997,"Exorcist, The","[Horror, Mystery]",1973,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1611,2153,"Avengers, The","[Action, Adventure]",1998,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2547,3409,Final Destination,"[Drama, Thriller]",2000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2636,3527,Predator,"[Action, Sci-Fi, Thriller]",1987,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3051,4085,Beverly Hills Cop,"[Action, Comedy, Crime, Drama]",1984,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6216,45662,"Omen, The","[Horror, Thriller]",2006,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7693,89745,"Avengers, The","[Action, Adventure, Sci-Fi, IMAX]",2012,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [73]:
Srija_genres.reset_index(drop=True, inplace=True) # resetting the index to default

# dropping attributes not required
Srija_genres.drop(['movieId','title','genres','year'], axis=1, inplace=True)

#checking if the drop was successful
Srija_genres

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [74]:
Srija_movie_ratings.shape

(9, 3)

In [75]:
Srija_genres.shape

(9, 20)

In [77]:
# making Srija's profile by doing a dot product between srija's prefered genre and srija's ratings for that genre  

Srija_profile = Srija_genres.T.dot(Srija_movie_ratings.rating)
Srija_profile
# we can see Srija prefers triller action and horror more so now we make a recommendation in these genres

Adventure              7.8
Animation              0.0
Children               0.0
Comedy                 8.8
Fantasy                0.0
Romance                3.9
Drama                 13.3
Action                17.2
Crime                  4.9
Thriller              18.9
Horror                14.9
Mystery               10.0
Sci-Fi                 7.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   3.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [79]:
movies_with_genres = movies_with_genres.set_index(movies_with_genres.movieId) # setting movieId as index
movies_with_genres.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,,
1,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
# dropping unnecessary attributes
movies_with_genres.drop(['movieId','title','genres','year'], axis=1, inplace=True)
movies_with_genres.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
# Taking a weighted average of all srija's preferd genre with movie genre table and recommending the top ones
recommendation_table = (movies_with_genres.dot(Srija_profile)) / Srija_profile.sum()
recommendation_table.head()

movieId
1    0.150635
2    0.070780
3    0.115245
4    0.235935
5    0.079855
dtype: float64

In [83]:
# choosing the top 20 recommendations
recommendation_table.sort_values(ascending=False, inplace=True) # sorting values from great to small
recommendation_table.head(20)

movieId
81132    0.869328
43932    0.742287
7235     0.707804
36509    0.692377
79132    0.678766
51545    0.663339
198      0.651543
74685    0.651543
6395     0.651543
54771    0.651543
26701    0.651543
4956     0.634301
60684    0.634301
4210     0.627949
71999    0.622505
31804    0.621597
91542    0.613430
72165    0.613430
2617     0.613430
27683    0.610708
dtype: float64

In [84]:
copy = movies.copy(deep=True)  # making copy of the original movies

copy = copy.set_index('movieId', drop=True)  # setting movieId as index

top_20_index = recommendation_table.index[:20].tolist() # top 20 recommended movieIds

# slice these indices from the copied movies and save in a variable
recommended_movies = copy.loc[top_20_index, :]

recommended_movies # displaying the 20 top recommendations

,title,genres,year
movieId,,,
81132,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film...",2010
43932,Pulse,"[Action, Drama, Fantasy, Horror, Mystery, Sci-...",2006
7235,Ichi the Killer (Koroshiya 1),"[Action, Comedy, Crime, Drama, Horror, Thriller]",2001
36509,"Cave, The","[Action, Adventure, Horror, Mystery, Sci-Fi, T...",2005
79132,Inception,"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill...",2010
51545,Pusher III: I'm the Angel of Death,"[Action, Comedy, Drama, Horror, Thriller]",2005
198,Strange Days,"[Action, Crime, Drama, Mystery, Sci-Fi, Thriller]",1995
74685,"Crazies, The","[Action, Drama, Horror, Sci-Fi, Thriller]",2010
6395,"Crazies, The (a.k.a. Code Name: Trixie)","[Action, Drama, Horror, Sci-Fi, Thriller]",1973
